Implementing a simple inverted index
====================================

In this exercise you will implement a simple inverted index in python.

We provide a boolean query parser for you. You can use the parser as follows

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
from queryparser import parse_query
parse_query('Caesar AND Brutus')

Note that the result of `parse_query` is an abstract syntax tree (AST).  We
provide another method `process_ast`, which will traverse the AST and produce
a flattened version for some queries.  The parser can deal with any valid
boolean query which uses the operators `AND`, `OR`, and `NOT`. However your
solution will not have to be able to handle all the possible queries.

In [ ]:
from queryparser import parse_query, process_ast
process_ast(parse_query('Caesar AND Brutus'))

To handle errors in the query parser, you can catch `ParseException`.  For
example, the query `Clown AND OR Circus` is not a well-formed boolean query.
To not have your code crash with an unhandled exception you can do something
like the following:

In [ ]:
from queryparser import ParseException
try:
    ast = parse_query('Clown AND OR Circus')
except ParseException as e:
    print("Failed to parse query:\n", e)

For the purposes of this exercise we provide you with the collected works of
Shakespeare as retrieved from
[gutenberg.org](http://www.gutenberg.org/files/100/100-0.txt) and split up
into individual text files which you can find in `shared/corpus` in the root
directory of your Jupyter account (which is your home directory on that
machine).

In a future exercise, we will ask you to implement an algorithm for splitting
up such a text file into smaller documents yourselves, but for this week, we
have done the preprocessing for you.

To iterate over all the text files, for example to add them to your inverted
index, you can use python's `glob` library which allows you to list files
using regular Unix shell [glob syntax](https://en.wikipedia.org/wiki/Glob_(programming).

In [ ]:
import glob
for file in glob.glob("../../shared/corpus/*.txt"):
    print(file)

Now that we have a way to parse a boolean query, and a corpus of documents,
what is left to do is to implement a simple inverted index.  For this exercise
you can assume that the dictionary, and the postings fit into main memory.

We also provide you with a very simple method which handles both the
tokenizing of a document and preprocessing of the resulting tokens.  This
method will not do much linguistic preprocessing but rather just strips each
token of special characters such as quotes, punctuation marks, etc.

We will discuss linguistic preprocessing in more details in future exercises.

Below is a very simple example of how to use the `tokenize_document` function.

In [ ]:
from textutils import tokenize_document
for token in tokenize_document('../../shared/corpus/the_phoenix_and_the_turtle.txt'):
    print(token)

Now all that is left to do is to implement some code which will create an
inverted index.  We give you a very simple skeleton which outlines the
implementation work that you will have to do.

In the index we want to refer to documents by a document ID.
We define variables `documents` which we will assume to be the datastructure
mapping document IDs to documents and `the_index` which will be the variable
holding the inverted index.

Note that while we currently initialize `the_index` to `None`, you will have
to choose a data structure to represent the inverted index, and initialize the
variable accordingly.

In [ ]:
documents = dict()
the_index = None

The easiest way to do this for now, is to simply have a counter which we
increment every time we add a document to the index.  In order to be able to
produce human-readable results you will need to also keep track of which
document ID refers to which document.  The function we provide to print
results expects that `documents` is a `dict` mapping document IDs to document
file names.

In [ ]:
documentid_counter = 1
def add_document(path):
    '''
    Add a document to the inverted index. Return the document's document ID.
    Remember the mapping from document ID to document in the `documents`
    data structure.
    '''
    # make sure that we access the global variables we have defined
    global the_index, documents, documentid_counter
    print("Adding '%s' to index" % path)

Assuming you have implemented `add_document` correctly you should be able to
create an inverted index as follows.

In [ ]:
import glob
for file in glob.glob('../../shared/corpus/*.txt'):
    add_document(file)

Now you need to implement a method to process boolean queries.  For this
exercise it is enough if your query processing can handle simple boolean
query with at most one operation.  Below is a selection of queries your
implementation should be able to handle.

So you do not have to walk the AST you can use the method `process_ast` which
was mentioned earlier to transform the AST into a flattened representation.
Additionally we provide a method `operation_is_complex`, which will return
`True` if the query is more complex than what we reasonably expect you to
handle.

In [ ]:
from queryparser import process_ast
ast = parse_query('Brutus AND Calpurnia')
print("AST representation:", ast)
flat = process_ast(ast)
print("Flattened representation:", flat)
print(flat.op)
print(flat.args)

As you can see, it is easier to extract the operation and operands from the
flat representation.  The flattened representation is composed from
`Operation` objects, and strings.
An `Operation` object is a representation of a boolean query operation.  The
`op` field represents the operation type which should be one of `AND`, `OR`,
`NOT`, or `LOOKUP`.
The `args` field is a list of one or more arguments for the operation.  `NOT`
and `LOOKUP` operations are expected to have a argument list of length one.
`AND`, and `OR` can have arbitrarily long argument lists, but the argument
list for such an operation should contain at least two elements.
For `AND` and `OR`, we represent operands of the form 'NOT <term>' as the
string '-<term>' to allow more efficient execution of queries of the form 'a
AND NOT b' and 'a OR NOT b'.

Before directly diving into the full query processing code, you will have to
implement methods that compute the intersection and union of two postings
lists.  We give you method declarations for these operations, but if you need
to change the method arguments feel free to do so.

While we do not give hard requirements on the method arguments for either
operation, you should probably consider making it a requirement that the
postings lists which you pass to `intersect` and `union` are sorted, as that
will make implementing the functionality significantly easier.

In [ ]:
def intersect(p1, p2):
    '''
    Method to compute the intersection of two postings lists.  Takes two
    postings lists as arguments and returns the intersection.
    '''
    pass

def union(p1, p2):
    '''
    Method to compute the union of two postings lists.  Takes two
    postings lists as arguments and returns the union.
    '''
    pass

Below there are a few example calls to `intersect` and `union` which you can
use to test your implementation.

In [ ]:
print(intersect([1,2,3,6], [1,3,4])) # should produce [1,3]
print(intersect([1,6], [3,4])) # should produce []
print(intersect([1,2,6], [1,6])) # should produce [1,6]
print(intersect([], [1,3])) # should produce []

print(union([1,2], [3,4])) # should produce [1,2,3,4]
print(union([1,2], [1,2,4])) # should produce [1,2,4]
print(union([1,2,6], [2,4,5])) # should produce [1,2,4,5,6]
print(union([], [1,3])) # should produce [1,3]

Now you should be fairly well-equipped to implement a boolean query processor.
Feel free to split your implementation into multiple methods if you find that
`execute_query` is getting too big for your taste.

Note that we expect you to implement intersections and unions of more than two
terms in the way discussed in the lecture and the book.

Queries that are conjunctions where all terms are negated (i.e.
`NOT Brutus AND NOT Calpurnia`) and queries that are disjunction where
at least one term is negated (i.e. `Brutus OR NOT Calpurnia OR Hamlet`)
are often not implemented in real-world, large-scale, general purpose systems.
We do not expect you to implement them either, your code can just return `None`
in these cases.

In [ ]:
from queryparser import operation_is_complex
def execute_query(query):
    '''
    Execute a boolean query on the inverted index. This method should return a
    list of document ids which satisfy the query. The list does not need to be
    in a particular order.
    '''
    try:
        ast = parse_query(query)
    except ParseException as e:
        print("Failed to parse query '%s':\n" % query, e)

    # flatten query AST
    flat = process_ast(ast)

    if operation_is_complex(flat):
        print("Complex query '%s'" % query)
        return None

    # TODO: implement query processing
    print("Processing", flat)

If you can process all the queries given below, you have made a good start on
building an inverted index and boolean query processor.
A good way to improve your query processing is to directly handle `AND NOT`.
You can either try to extend the `intersect` method which you implemented
previously, or you can implement `AND NOT` in a separate method.

In order to get human readable output, the following function translates
document IDs back to file names, assuming you've correctly populated the
`documents` dict.

In [ ]:
def print_result(docs):
    if not docs:
        print("No documents found")
        print()
        return
    # If we got some results, print them
    for doc in docs:
        print(documents[doc])
    print()

In [ ]:
print_result(execute_query('Caesar'))
print_result(execute_query('hello'))
print_result(execute_query('Brutus AND Calpurnia'))
print_result(execute_query('Brutus OR Hamlet'))
print_result(execute_query('Brutus AND NOT Calpurnia'))
print_result(execute_query('Caesar AND Brutus AND Calpurnia'))
print_result(execute_query('Caesar AND Brutus AND NOT Calpurnia'))

If you wish, you can try to improve your query processor to handle more
complex queries, such as the examples below.  This part is optional.

Note that the query parser gives `NOT` precedence over `AND`, and `AND`
precedence over `OR` in the absence of parentheses in queries.

In [ ]:
print_result(execute_query('(Caesar OR Brutus) AND Calpurnia'))
print_result(execute_query('(Caesar OR Brutus) AND (Calpurnia OR clown)'))
print_result(execute_query('(Brutus AND Calpurnia) OR (Romeo AND Juliet)'))
print_result(execute_query('Caesar OR Brutus AND Calpurnia'))

If everything so far was a walk in the park for you, you can even try to
implement a method which converts any query into conjunctive normal form
(CNF).  Note, however, that converting a boolean formula to CNF is NP hard,
and as such we do not require you to do this.